# Blog App API Endpoints Testing

This notebook provides comprehensive testing of all Blog app endpoints in the FastAPI DDD project.

## 📚 What You'll Learn

- **Authentication**: How to authenticate as admin user
- **Categories API**: Full CRUD operations for blog categories
- **Tags API**: Complete tag management functionality
- **Posts API**: Blog post creation, reading, updating, and deletion
- **Comments API**: Comment system with moderation features
- **Statistics API**: Blog analytics and metrics (admin only)
- **Health Check**: System status verification

## 🛠️ Prerequisites

1. **FastAPI server running**: `make dev` (http://localhost:8000)
2. **Database initialized**: `make db-init`
3. **Admin user exists**: Check settings for `FIRST_SUPERUSER`

## 🚀 Getting Started

Run each cell in order to test the complete Blog app functionality.

In [16]:
# Import Required Libraries
import requests
import json
import uuid
import time
from datetime import datetime, timezone
from typing import Dict, Any, Optional
from pprint import pprint

# Configuration
print("📦 Libraries imported successfully!")
print("🔧 Ready to test Blog app endpoints")

📦 Libraries imported successfully!
🔧 Ready to test Blog app endpoints


In [ ]:
# Define Base URL and Configuration
BASE_URL = "http://localhost:8000"
API_V1_URL = f"{BASE_URL}/api/v1"
BLOG_API_URL = f"{API_V1_URL}/blog"

# Admin credentials (update these to match your settings)
ADMIN_EMAIL = "admin@example.com"
ADMIN_PASSWORD = "ChangeThis123!"

# Global variables for storing data
access_token = None
headers = {}
created_data = {"categories": [], "tags": [], "posts": [], "comments": []}

print(f"🌐 Base URL: {BASE_URL}")
print(f"🔗 Blog API URL: {BLOG_API_URL}")
print(f"👤 Admin User: {ADMIN_EMAIL}")

🌐 Base URL: http://localhost:8000
🔗 Blog API URL: http://localhost:8000/api/v1/blog
👤 Admin User: admin@example.com


## 📋 Complete List of Blog App Endpoints

The Blog app provides a comprehensive REST API with the following endpoints:

### 🏥 Health Check
- `GET /api/v1/blog/health` - Blog app health status

### 📁 Categories API
- `POST /api/v1/blog/categories/` - Create category (admin only)
- `GET /api/v1/blog/categories/` - List all categories (public)
- `GET /api/v1/blog/categories/{category_id}` - Get category by ID (public)
- `PUT /api/v1/blog/categories/{category_id}` - Update category (admin only)
- `DELETE /api/v1/blog/categories/{category_id}` - Delete category (admin only)

### 🏷️ Tags API
- `POST /api/v1/blog/tags/` - Create tag (admin only)
- `GET /api/v1/blog/tags/` - List all tags (public)
- `GET /api/v1/blog/tags/{tag_id}` - Get tag by ID (public)

### 📝 Posts API
- `POST /api/v1/blog/posts/` - Create blog post (authenticated)
- `GET /api/v1/blog/posts/` - List posts with filtering (public)
- `GET /api/v1/blog/posts/{post_slug}` - Get post by slug (public)
- `PUT /api/v1/blog/posts/{post_id}` - Update post (author/admin)
- `DELETE /api/v1/blog/posts/{post_id}` - Delete post (author/admin)

### 💬 Comments API
- `POST /api/v1/blog/comments/` - Create comment (public)
- `GET /api/v1/blog/comments/` - List comments with filtering (public)
- `PUT /api/v1/blog/comments/{comment_id}/moderate` - Moderate comment (admin only)
- `DELETE /api/v1/blog/comments/{comment_id}` - Delete comment (admin only)

### 📊 Statistics API
- `GET /api/v1/blog/stats/` - Get blog statistics (admin only)

In [19]:
# Helper Functions for API Testing


def make_request(method: str, url: str, **kwargs) -> Dict[str, Any]:
    """Make HTTP request and return structured response"""
    try:
        response = requests.request(method, url, **kwargs)
        return {
            "status_code": response.status_code,
            "success": response.status_code < 400,
            "data": response.json() if response.text else None,
            "headers": dict(response.headers),
            "url": response.url,
        }
    except requests.exceptions.RequestException as e:
        return {"status_code": 0, "success": False, "error": str(e), "url": url}
    except json.JSONDecodeError:
        return {
            "status_code": response.status_code,
            "success": response.status_code < 400,
            "data": response.text,
            "headers": dict(response.headers),
            "url": response.url,
        }


def print_response(response: Dict[str, Any], title: str = "") -> None:
    """Pretty print API response"""
    if title:
        print(f"\n🔍 {title}")
        print("=" * (len(title) + 4))

    status_icon = "✅" if response["success"] else "❌"
    print(f"{status_icon} Status: {response['status_code']}")

    if "error" in response:
        print(f"❌ Error: {response['error']}")
    elif response["data"]:
        if isinstance(response["data"], dict):
            print("📄 Response Data:")
            pprint(response["data"])
        else:
            print(f"📄 Response: {response['data']}")
    print()


def generate_unique_slug(base: str) -> str:
    """Generate unique slug with timestamp"""
    timestamp = int(time.time())
    return f"{base}-{timestamp}"


print("🛠️ Helper functions defined successfully!")

🛠️ Helper functions defined successfully!


## 🔐 Step 1: Authentication

First, we need to authenticate as an admin user to access protected endpoints.

In [20]:
# Authenticate Admin User
def authenticate_admin() -> bool:
    """Authenticate admin user and get access token"""
    global access_token, headers

    login_url = f"{API_V1_URL}/auth/login/access-token"
    login_data = {"username": ADMIN_EMAIL, "password": ADMIN_PASSWORD}

    print("🔐 Authenticating admin user...")
    response = make_request("POST", login_url, data=login_data)
    print_response(response, "Admin Authentication")

    if response["success"] and response["data"]:
        access_token = response["data"].get("access_token")
        if access_token:
            headers = {
                "Authorization": f"Bearer {access_token}",
                "Content-Type": "application/json",
            }
            print(f"✅ Successfully authenticated as: {ADMIN_EMAIL}")
            print(f"🔑 Access token obtained (length: {len(access_token)} chars)")
            return True

    print("❌ Authentication failed!")
    print("💡 Make sure:")
    print("   - Server is running: make dev")
    print("   - Database is initialized: make db-init")
    print("   - Admin credentials are correct")
    return False


# Run authentication
success = authenticate_admin()
if not success:
    print("⚠️ Cannot continue without authentication")

🔐 Authenticating admin user...

🔍 Admin Authentication
❌ Status: 400
📄 Response Data:
{'detail': 'Incorrect email or password'}

❌ Authentication failed!
💡 Make sure:
   - Server is running: make dev
   - Database is initialized: make db-init
   - Admin credentials are correct
⚠️ Cannot continue without authentication

🔍 Admin Authentication
❌ Status: 400
📄 Response Data:
{'detail': 'Incorrect email or password'}

❌ Authentication failed!
💡 Make sure:
   - Server is running: make dev
   - Database is initialized: make db-init
   - Admin credentials are correct
⚠️ Cannot continue without authentication


## 🏥 Step 2: Health Check

Test the health check endpoint to verify the Blog app is running correctly.

In [21]:
# Test Health Check Endpoint
def test_health_check():
    """Test blog app health check"""
    url = f"{BLOG_API_URL}/health"
    response = make_request("GET", url)
    print_response(response, "Blog App Health Check")
    return response["success"]


# Run health check
health_ok = test_health_check()
if health_ok:
    print("🎉 Blog app is healthy and ready!")
else:
    print("⚠️ Blog app health check failed")


🔍 Blog App Health Check
✅ Status: 200
📄 Response Data:
{'app': 'blog', 'status': 'healthy'}

🎉 Blog app is healthy and ready!


## 📁 Step 3: Categories API Testing

Test all category-related endpoints including CRUD operations.

In [ ]:
# Test Categories API
def test_categories_api():
    """Test all category endpoints"""
    print("🏗️ Testing Categories API...\n")

    # 1. Create Category (POST)
    print("📝 Creating new category...")
    category_data = {
        "name": "Technology",
        "slug": generate_unique_slug("technology"),
        "description": "Technology and programming posts",
    }

    url = f"{BLOG_API_URL}/categories/"
    response = make_request("POST", url, json=category_data, headers=headers)
    print_response(response, "Create Category")

    category_id = None
    if response["success"] and response["data"]:
        category_id = response["data"].get("id")
        created_data["categories"].append(response["data"])
        print(f"✅ Category created with ID: {category_id}")

    # 2. List Categories (GET)
    print("\n📋 Listing all categories...")
    response = make_request("GET", url)
    print_response(response, "List Categories")

    if response["success"] and response["data"]:
        total_categories = response["data"].get("total", 0)
        print(f"📊 Total categories: {total_categories}")

    # 3. Get Category by ID (GET)
    if category_id:
        print(f"\n🔍 Getting category by ID: {category_id}")
        url_with_id = f"{BLOG_API_URL}/categories/{category_id}"
        response = make_request("GET", url_with_id)
        print_response(response, "Get Category by ID")

    # 4. Update Category (PUT)
    if category_id:
        print(f"\n✏️ Updating category: {category_id}")
        update_data = {
            "name": "Advanced Technology",
            "description": "Advanced technology and programming tutorials",
        }
        response = make_request("PUT", url_with_id, json=update_data, headers=headers)
        print_response(response, "Update Category")

    # 5. Create Parent-Child Categories
    print("\n👨‍👩‍👧‍👦 Creating parent-child category relationship...")
    child_category_data = {
        "name": "Python Programming",
        "slug": generate_unique_slug("python-programming"),
        "description": "Python programming tutorials",
        "parent_id": category_id,
    }

    response = make_request("POST", url, json=child_category_data, headers=headers)
    print_response(response, "Create Child Category")

    if response["success"] and response["data"]:
        created_data["categories"].append(response["data"])

    return category_id


# Run categories testing
test_category_id = test_categories_api()

🏗️ Testing Categories API...

📝 Creating new category...

🔍 Create Category
❌ Status: 401
📄 Response Data:
{'detail': 'Not authenticated'}


📋 Listing all categories...

🔍 List Categories
✅ Status: 200
📄 Response Data:
{'items': [{'created_at': '2025-07-31T10:31:19.037989',
            'description': 'Technology posts created via API',
            'id': '0bfcc964-3406-46b4-b3e7-50524219ba66',
            'is_active': True,
            'name': 'API Technology 1753957879',
            'parent_id': None,
            'post_count': 0,
            'slug': 'api-technology-1753957879',
            'updated_at': '2025-07-31T10:31:19.037995'},
           {'created_at': '2025-07-31T10:31:45.460421',
            'description': 'Technology posts created via API',
            'id': '7a45ed27-bf21-4fe0-97b9-8689fc27c84e',
            'is_active': True,
            'name': 'API Technology 1753957905',
            'parent_id': None,
            'post_count': 0,
            'slug': 'api-technology-17539

## 🏷️ Step 4: Tags API Testing

Test all tag-related endpoints for creating and managing blog tags.

In [ ]:
# Test Tags API
def test_tags_api():
    """Test all tag endpoints"""
    print("🏷️ Testing Tags API...\n")

    # 1. Create Tag (POST)
    print("📝 Creating new tag...")
    tag_data = {
        "name": "Python",
        "slug": generate_unique_slug("python"),
        "description": "Python programming language",
    }

    url = f"{BLOG_API_URL}/tags/"
    response = make_request("POST", url, json=tag_data, headers=headers)
    print_response(response, "Create Tag")

    tag_id = None
    if response["success"] and response["data"]:
        tag_id = response["data"].get("id")
        created_data["tags"].append(response["data"])
        print(f"✅ Tag created with ID: {tag_id}")

    # 2. Create Multiple Tags
    print("\n📝 Creating additional tags...")
    additional_tags = [
        {
            "name": "FastAPI",
            "slug": generate_unique_slug("fastapi"),
            "description": "FastAPI web framework",
        },
        {
            "name": "Web Development",
            "slug": generate_unique_slug("web-development"),
            "description": "Web development technologies",
        },
    ]

    for tag in additional_tags:
        response = make_request("POST", url, json=tag, headers=headers)
        if response["success"] and response["data"]:
            created_data["tags"].append(response["data"])
            print(f"✅ Created tag: {tag['name']}")

    # 3. List Tags (GET)
    print("\n📋 Listing all tags...")
    response = make_request("GET", url)
    print_response(response, "List Tags")

    if response["success"] and response["data"]:
        total_tags = response["data"].get("total", 0)
        print(f"📊 Total tags: {total_tags}")

    # 4. Get Tag by ID (GET)
    if tag_id:
        print(f"\n🔍 Getting tag by ID: {tag_id}")
        url_with_id = f"{BLOG_API_URL}/tags/{tag_id}"
        response = make_request("GET", url_with_id)
        print_response(response, "Get Tag by ID")

    return tag_id


# Run tags testing
test_tag_id = test_tags_api()

🏷️ Testing Tags API...

📝 Creating new tag...

🔍 Create Tag
❌ Status: 401
📄 Response Data:
{'detail': 'Not authenticated'}


📝 Creating additional tags...

📋 Listing all tags...

🔍 List Tags
✅ Status: 200
📄 Response Data:
{'items': [{'created_at': '2025-07-31T10:31:19.043917',
            'description': 'Tag for API testing posts',
            'id': '49251abb-8e86-4698-a623-3bc8850ff932',
            'is_active': True,
            'name': 'API Testing 1753957879',
            'post_count': 0,
            'slug': 'api-testing-1753957879'},
           {'created_at': '2025-07-31T10:31:45.466088',
            'description': 'Tag for API testing posts',
            'id': 'ce066fb4-457d-41c3-a6b9-cadf40d086ba',
            'is_active': True,
            'name': 'API Testing 1753957905',
            'post_count': 0,
            'slug': 'api-testing-1753957905'},
           {'created_at': '2025-07-31T10:31:45.172220',
            'description': 'Python programming for demo',
            'id': 

## 📝 Step 5: Blog Posts API Testing

Test all blog post endpoints including creation, reading, updating, and deletion.

In [ ]:
# Test Blog Posts API
def test_posts_api():
    """Test all blog post endpoints"""
    print("📝 Testing Blog Posts API...\n")

    # 1. Create Blog Post (POST)
    print("📝 Creating new blog post...")
    post_data = {
        "title": "Getting Started with FastAPI and DDD",
        "slug": generate_unique_slug("getting-started-fastapi-ddd"),
        "content": """# Getting Started with FastAPI and DDD

FastAPI is a modern, fast web framework for building APIs with Python 3.7+ based on standard Python type hints.

## Why FastAPI?

- **Fast**: Very high performance, on par with NodeJS and Go
- **Fast to code**: Increase the speed to develop features
- **Fewer bugs**: Reduce about 40% of human (developer) induced errors
- **Intuitive**: Great editor support with completion everywhere
- **Easy**: Designed to be easy to use and learn
- **Short**: Minimize code duplication
- **Robust**: Get production-ready code with automatic interactive documentation
- **Standards-based**: Based on open standards for APIs

## Domain-Driven Design (DDD)

DDD is a software development approach that focuses on understanding the business domain and modeling software accordingly.

### Key Benefits:
1. Better communication between technical and business teams
2. More maintainable and flexible code
3. Clear separation of concerns
4. Easier to test and debug

This blog post demonstrates how to combine FastAPI with DDD principles for building robust APIs.""",
        "excerpt": "Learn how to build modern APIs using FastAPI with Domain-Driven Design principles.",
        "status": "published",
        "category_id": test_category_id if test_category_id else None,
        "published_at": datetime.now(timezone.utc).isoformat(),
    }

    url = f"{BLOG_API_URL}/posts/"
    response = make_request("POST", url, json=post_data, headers=headers)
    print_response(response, "Create Blog Post")

    post_id = None
    post_slug = None
    if response["success"] and response["data"]:
        post_id = response["data"].get("id")
        post_slug = response["data"].get("slug")
        created_data["posts"].append(response["data"])
        print(f"✅ Blog post created with ID: {post_id}")
        print(f"🔗 Post slug: {post_slug}")

    # 2. Create Draft Post
    print("\n📝 Creating draft blog post...")
    draft_data = {
        "title": "Advanced FastAPI Patterns (Draft)",
        "slug": generate_unique_slug("advanced-fastapi-patterns"),
        "content": "This is a draft post about advanced FastAPI patterns...",
        "excerpt": "Advanced patterns for FastAPI development",
        "status": "draft",
        "category_id": test_category_id if test_category_id else None,
    }

    response = make_request("POST", url, json=draft_data, headers=headers)
    if response["success"] and response["data"]:
        created_data["posts"].append(response["data"])
        print(f"✅ Draft post created: {draft_data['title']}")

    # 3. List Blog Posts (GET)
    print("\n📋 Listing all blog posts...")
    response = make_request("GET", url)
    print_response(response, "List Blog Posts")

    if response["success"] and response["data"]:
        total_posts = response["data"].get("total", 0)
        print(f"📊 Total posts: {total_posts}")

    # 4. List Published Posts Only
    print("\n📋 Listing published posts only...")
    params = {"published_only": "true"}
    response = make_request("GET", url, params=params)
    print_response(response, "List Published Posts")

    # 5. Filter Posts by Category
    if test_category_id:
        print(f"\n🔍 Filtering posts by category: {test_category_id}")
        params = {"category_id": test_category_id}
        response = make_request("GET", url, params=params)
        print_response(response, "Filter Posts by Category")

    # 6. Search Posts
    print("\n🔍 Searching posts with keyword 'FastAPI'...")
    params = {"search": "FastAPI"}
    response = make_request("GET", url, params=params)
    print_response(response, "Search Posts")

    # 7. Get Post by Slug (GET)
    if post_slug:
        print(f"\n🔍 Getting post by slug: {post_slug}")
        url_with_slug = f"{BLOG_API_URL}/posts/{post_slug}"
        response = make_request("GET", url_with_slug)
        print_response(response, "Get Post by Slug")

        # Test view count increment
        print("\n👁️ Testing view count increment...")
        response = make_request("GET", url_with_slug)
        if response["success"] and response["data"]:
            view_count = response["data"].get("view_count", 0)
            print(f"📊 Current view count: {view_count}")

    # 8. Update Blog Post (PUT)
    if post_id:
        print(f"\n✏️ Updating blog post: {post_id}")
        update_data = {
            "title": "Getting Started with FastAPI and DDD - Updated",
            "excerpt": "Updated: Learn how to build modern APIs using FastAPI with Domain-Driven Design principles.",
            "content": post_data["content"]
            + "\n\n## Update\n\nThis post has been updated with additional information.",
        }
        url_with_id = f"{BLOG_API_URL}/posts/{post_id}"
        response = make_request("PUT", url_with_id, json=update_data, headers=headers)
        print_response(response, "Update Blog Post")

    return post_id, post_slug


# Run posts testing
test_post_id, test_post_slug = test_posts_api()

📝 Testing Blog Posts API...

📝 Creating new blog post...

🔍 Create Blog Post
❌ Status: 401
📄 Response Data:
{'detail': 'Not authenticated'}


📝 Creating draft blog post...

📋 Listing all blog posts...

🔍 List Blog Posts
✅ Status: 200
📄 Response Data:
{'items': [{'author_id': 'd5418109-8570-4f28-a33a-798e34164ab8',
            'category': None,
            'category_id': None,
            'comment_count': 0,
            'content': 'This post was created via the API to demonstrate '
                       'functionality.',
            'created_at': '2025-07-31T10:31:45.472605',
            'excerpt': 'API demo post excerpt',
            'featured_image': None,
            'id': '2dc85800-3148-4041-a40f-548ddd5ee7a6',
            'is_active': True,
            'meta_description': None,
            'meta_title': None,
            'published_at': '2025-07-31T12:31:45.472561',
            'slug': 'api-demo-post-1753957905',
            'status': 'published',
            'tags': [],
         

## 💬 Step 6: Comments API Testing

Test the comments system including creation, listing, and moderation features.

In [ ]:
# Test Comments API
def test_comments_api():
    """Test all comment endpoints"""
    print("💬 Testing Comments API...\n")

    if not test_post_id:
        print("⚠️ No test post available for commenting")
        return None

    # 1. Create Comment (POST) - Public endpoint
    print("📝 Creating new comment...")
    comment_data = {
        "content": "Great post! Very informative and well-written. Thanks for sharing these insights about FastAPI and DDD.",
        "author_name": "John Developer",
        "author_email": "john.dev@example.com",
        "post_id": test_post_id,
    }

    url = f"{BLOG_API_URL}/comments/"
    response = make_request("POST", url, json=comment_data)
    print_response(response, "Create Comment")

    comment_id = None
    if response["success"] and response["data"]:
        comment_id = response["data"].get("id")
        created_data["comments"].append(response["data"])
        print(f"✅ Comment created with ID: {comment_id}")

    # 2. Create Reply Comment
    print("\n💬 Creating reply comment...")
    reply_data = {
        "content": "I agree! FastAPI is amazing for building APIs quickly.",
        "author_name": "Jane Smith",
        "author_email": "jane.smith@example.com",
        "post_id": test_post_id,
        "parent_id": comment_id if comment_id else None,
    }

    response = make_request("POST", url, json=reply_data)
    if response["success"] and response["data"]:
        created_data["comments"].append(response["data"])
        print(f"✅ Reply comment created")

    # 3. Create Multiple Comments
    print("\n📝 Creating additional comments...")
    additional_comments = [
        {
            "content": "This tutorial helped me understand DDD principles better.",
            "author_name": "Alice Johnson",
            "author_email": "alice.j@example.com",
            "post_id": test_post_id,
        },
        {
            "content": "Looking forward to more posts about FastAPI!",
            "author_name": "Bob Wilson",
            "author_email": "bob.w@example.com",
            "post_id": test_post_id,
        },
    ]

    for comment in additional_comments:
        response = make_request("POST", url, json=comment)
        if response["success"] and response["data"]:
            created_data["comments"].append(response["data"])
            print(f"✅ Created comment by: {comment['author_name']}")

    # 4. List All Comments (GET)
    print("\n📋 Listing all comments...")
    response = make_request("GET", url)
    print_response(response, "List All Comments")

    if response["success"] and response["data"]:
        total_comments = response["data"].get("total", 0)
        print(f"📊 Total comments: {total_comments}")

    # 5. Filter Comments by Post
    print(f"\n🔍 Filtering comments for post: {test_post_id}")
    params = {"post_id": test_post_id}
    response = make_request("GET", url, params=params)
    print_response(response, "Filter Comments by Post")

    # 6. Filter Comments by Status
    print("\n🔍 Filtering comments by status (pending)...")
    params = {"status": "pending"}
    response = make_request("GET", url, params=params)
    print_response(response, "Filter Comments by Status")

    # 7. Moderate Comment (PUT) - Admin only
    if comment_id:
        print(f"\n✅ Moderating comment: {comment_id}")
        moderate_url = f"{BLOG_API_URL}/comments/{comment_id}/moderate"
        moderate_data = {
            "status": "approved",
            "moderator_note": "Comment approved - follows community guidelines",
        }
        response = make_request(
            "PUT", moderate_url, json=moderate_data, headers=headers
        )
        print_response(response, "Moderate Comment")

    return comment_id


# Run comments testing
test_comment_id = test_comments_api()

💬 Testing Comments API...

⚠️ No test post available for commenting


## 📊 Step 7: Statistics API Testing

Test the blog statistics endpoint (admin only) to get analytics and metrics.

In [ ]:
# Test Statistics API
def test_statistics_api():
    """Test blog statistics endpoint (admin only)"""
    print("📊 Testing Statistics API...\n")

    # Get Blog Statistics (GET) - Admin only
    print("📈 Getting blog statistics...")
    url = f"{BLOG_API_URL}/stats/"
    response = make_request("GET", url, headers=headers)
    print_response(response, "Blog Statistics")

    if response["success"] and response["data"]:
        stats = response["data"]
        print("📊 Blog Statistics Summary:")
        print(f"   📝 Total Posts: {stats.get('total_posts', 0)}")
        print(f"   📄 Published Posts: {stats.get('published_posts', 0)}")
        print(f"   📝 Draft Posts: {stats.get('draft_posts', 0)}")
        print(f"   💬 Total Comments: {stats.get('total_comments', 0)}")
        print(f"   ✅ Approved Comments: {stats.get('approved_comments', 0)}")
        print(f"   ⏳ Pending Comments: {stats.get('pending_comments', 0)}")
        print(f"   📁 Total Categories: {stats.get('total_categories', 0)}")
        print(f"   🏷️ Total Tags: {stats.get('total_tags', 0)}")
        print(f"   👁️ Total Views: {stats.get('total_views', 0)}")

    # Test without authentication (should fail)
    print("\n🔒 Testing statistics access without authentication...")
    response = make_request("GET", url)  # No headers
    if not response["success"]:
        print("✅ Correctly denied access without authentication")
    else:
        print("⚠️ Unexpected: Statistics accessible without authentication")


# Run statistics testing
test_statistics_api()

📊 Testing Statistics API...

📈 Getting blog statistics...

🔍 Blog Statistics
❌ Status: 401
📄 Response Data:
{'detail': 'Not authenticated'}


🔒 Testing statistics access without authentication...
✅ Correctly denied access without authentication


## 🧪 Step 8: Advanced Testing Scenarios

Test edge cases, error handling, and advanced features.

In [ ]:
# Advanced Testing Scenarios
def test_advanced_scenarios():
    """Test edge cases and advanced features"""
    print("🧪 Testing Advanced Scenarios...\n")

    # 1. Test Pagination
    print("📄 Testing pagination...")
    url = f"{BLOG_API_URL}/posts/"
    params = {"skip": 0, "limit": 2}
    response = make_request("GET", url, params=params)
    print_response(response, "Pagination Test")

    if response["success"] and response["data"]:
        print(
            f"📊 Page info: Page {response['data'].get('page', 1)} of {response['data'].get('pages', 1)}"
        )

    # 2. Test Invalid Data
    print("\n❌ Testing invalid post creation...")
    invalid_post_data = {
        "title": "",  # Empty title
        "slug": "invalid-slug!",  # Invalid characters
        "content": "",  # Empty content
    }

    response = make_request("POST", url, json=invalid_post_data, headers=headers)
    if not response["success"]:
        print("✅ Correctly rejected invalid post data")
        print(f"📄 Validation error: {response['status_code']}")

    # 3. Test Non-existent Resource
    print("\n🔍 Testing access to non-existent post...")
    fake_slug = "non-existent-post-12345"
    url_fake = f"{BLOG_API_URL}/posts/{fake_slug}"
    response = make_request("GET", url_fake)
    if response["status_code"] == 404:
        print("✅ Correctly returned 404 for non-existent post")

    # 4. Test Unauthorized Access
    print("\n🔒 Testing unauthorized access to admin endpoints...")
    url_categories = f"{BLOG_API_URL}/categories/"
    category_data = {
        "name": "Unauthorized Test",
        "slug": "unauthorized-test",
        "description": "This should fail",
    }

    # Request without authentication headers
    response = make_request("POST", url_categories, json=category_data)
    if not response["success"]:
        print("✅ Correctly denied unauthorized category creation")

    # 5. Test Duplicate Slug
    if created_data["posts"]:
        print("\n🔄 Testing duplicate slug creation...")
        existing_slug = created_data["posts"][0].get("slug")
        duplicate_post = {
            "title": "Duplicate Slug Test",
            "slug": existing_slug,  # Same slug as existing post
            "content": "This should fail due to duplicate slug",
        }

        response = make_request(
            "POST", f"{BLOG_API_URL}/posts/", json=duplicate_post, headers=headers
        )
        if not response["success"]:
            print("✅ Correctly prevented duplicate slug creation")

    # 6. Test Comment on Non-existent Post
    print("\n💬 Testing comment on non-existent post...")
    fake_post_comment = {
        "content": "This comment should fail",
        "author_name": "Test User",
        "author_email": "test@example.com",
        "post_id": "00000000-0000-0000-0000-000000000000",  # Fake UUID
    }

    response = make_request("POST", f"{BLOG_API_URL}/comments/", json=fake_post_comment)
    if not response["success"]:
        print("✅ Correctly prevented comment on non-existent post")

    print("\n🎉 Advanced testing scenarios completed!")


# Run advanced testing
test_advanced_scenarios()

🧪 Testing Advanced Scenarios...

📄 Testing pagination...

🔍 Pagination Test
✅ Status: 200
📄 Response Data:
{'items': [{'author_id': 'd5418109-8570-4f28-a33a-798e34164ab8',
            'category': None,
            'category_id': None,
            'comment_count': 0,
            'content': 'This post was created via the API to demonstrate '
                       'functionality.',
            'created_at': '2025-07-31T10:31:45.472605',
            'excerpt': 'API demo post excerpt',
            'featured_image': None,
            'id': '2dc85800-3148-4041-a40f-548ddd5ee7a6',
            'is_active': True,
            'meta_description': None,
            'meta_title': None,
            'published_at': '2025-07-31T12:31:45.472561',
            'slug': 'api-demo-post-1753957905',
            'status': 'published',
            'tags': [],
            'title': 'API Demo Post 1753957905',
            'updated_at': '2025-07-31T10:31:45.472607',
            'view_count': 0},
           {'autho

## 🧹 Step 9: Cleanup (Optional)

Clean up test data created during the testing process.

In [ ]:
# Cleanup Test Data
def cleanup_test_data():
    """Clean up test data created during testing"""
    print("🧹 Cleaning up test data...\n")

    # Delete Comments
    print("💬 Deleting test comments...")
    for comment in created_data["comments"]:
        comment_id = comment.get("id")
        if comment_id:
            url = f"{BLOG_API_URL}/comments/{comment_id}"
            response = make_request("DELETE", url, headers=headers)
            if response["success"]:
                print(f"✅ Deleted comment: {comment_id}")

    # Delete Posts
    print("\n📝 Deleting test posts...")
    for post in created_data["posts"]:
        post_id = post.get("id")
        if post_id:
            url = f"{BLOG_API_URL}/posts/{post_id}"
            response = make_request("DELETE", url, headers=headers)
            if response["success"]:
                print(f"✅ Deleted post: {post['title']}")

    # Delete Categories (Note: This might fail if categories have dependencies)
    print("\n📁 Deleting test categories...")
    # Delete child categories first, then parent categories
    categories_sorted = sorted(
        created_data["categories"], key=lambda x: x.get("parent_id") is not None
    )

    for category in categories_sorted:
        category_id = category.get("id")
        if category_id:
            url = f"{BLOG_API_URL}/categories/{category_id}"
            response = make_request("DELETE", url, headers=headers)
            if response["success"]:
                print(f"✅ Deleted category: {category['name']}")
            else:
                print(
                    f"⚠️ Could not delete category: {category['name']} (may have dependencies)"
                )

    print("\n🎉 Cleanup completed!")
    print("📊 Summary of created data:")
    print(f"   📁 Categories: {len(created_data['categories'])}")
    print(f"   🏷️ Tags: {len(created_data['tags'])}")
    print(f"   📝 Posts: {len(created_data['posts'])}")
    print(f"   💬 Comments: {len(created_data['comments'])}")


# Uncomment the next line to run cleanup
# cleanup_test_data()

print(
    "💡 To clean up test data, uncomment and run the cleanup_test_data() function above."
)

💡 To clean up test data, uncomment and run the cleanup_test_data() function above.


## 🎉 Testing Complete!

### 📋 Summary of Tested Endpoints

You have successfully tested all Blog app endpoints:

#### ✅ Health Check
- `GET /api/v1/blog/health` - Blog app status

#### ✅ Categories API (5 endpoints)
- `POST /api/v1/blog/categories/` - Create category
- `GET /api/v1/blog/categories/` - List categories
- `GET /api/v1/blog/categories/{id}` - Get category
- `PUT /api/v1/blog/categories/{id}` - Update category
- `DELETE /api/v1/blog/categories/{id}` - Delete category

#### ✅ Tags API (3 endpoints)
- `POST /api/v1/blog/tags/` - Create tag
- `GET /api/v1/blog/tags/` - List tags
- `GET /api/v1/blog/tags/{id}` - Get tag

#### ✅ Posts API (5 endpoints)
- `POST /api/v1/blog/posts/` - Create post
- `GET /api/v1/blog/posts/` - List posts (with filtering)
- `GET /api/v1/blog/posts/{slug}` - Get post by slug
- `PUT /api/v1/blog/posts/{id}` - Update post
- `DELETE /api/v1/blog/posts/{id}` - Delete post

#### ✅ Comments API (4 endpoints)
- `POST /api/v1/blog/comments/` - Create comment
- `GET /api/v1/blog/comments/` - List comments
- `PUT /api/v1/blog/comments/{id}/moderate` - Moderate comment
- `DELETE /api/v1/blog/comments/{id}` - Delete comment

#### ✅ Statistics API (1 endpoint)
- `GET /api/v1/blog/stats/` - Get blog statistics

### 🔧 Features Tested

- ✅ **Authentication** - Admin login and token usage
- ✅ **CRUD Operations** - Create, Read, Update, Delete for all entities
- ✅ **Filtering & Search** - Posts by category, tag, status, and keyword search
- ✅ **Pagination** - List endpoints with skip/limit parameters
- ✅ **Relationships** - Category hierarchies, post-comment relationships
- ✅ **Permissions** - Public vs authenticated vs admin-only endpoints
- ✅ **Error Handling** - Invalid data, non-existent resources, unauthorized access
- ✅ **Business Logic** - View count tracking, comment moderation, slug generation

### 🚀 Next Steps

1. **Explore the API**: Visit http://localhost:8000/docs for interactive documentation
2. **Build a Frontend**: Use these endpoints to build a blog frontend
3. **Extend Functionality**: Add file uploads, email notifications, or search features
4. **Monitor Performance**: Set up logging and monitoring for production use

### 📚 Additional Resources

- **FastAPI Documentation**: https://fastapi.tiangolo.com/
- **Domain-Driven Design**: Learn more about DDD patterns and practices
- **Testing Best Practices**: Implement automated testing for your endpoints

**Happy blogging! 🎉**